# data 불러오기

In [2]:
import pandas as pd
import re

In [4]:
data = pd.read_csv('everytime.csv', encoding='utf-8')

In [5]:
data2 = pd.read_csv('everytime_lecture.csv', encoding='utf-8')

# data 합치기

In [6]:
data2.columns = ['attendance', 'exam_times', 'grade', 'homework', 'lecture_name', 'professor_name', 'mean_rate', 'team']

In [7]:
data2 = data2.drop_duplicates(subset=['lecture_name', 'professor_name'])

In [1191]:
tot = pd.merge(data, data2, on=['lecture_name', 'professor_name'])

In [9]:
def filter_text(row):
    if len(row['text'])>=100:
        return row['text']
    else:
        return ''

In [ ]:
total = tot.sort_values(['lecture_name', 'professor_name'])

# text 추출

## text token 뽑아내기

In [ ]:
tokens = set()

for text in total['text']:
    token_list = twitter.pos(text)
    for token in token_list:
        tokens.add(token)

In [49]:
count = 0
for e in logest_text['text']:
    if '.' in e:
        count += 1
print(len(logest_text), '중에', count)

2381 중에 2089


In [54]:
count = 0
for e in logest_text['text']:
    if '요' in e:
        count += 1
print(len(logest_text), '중에', count)

2381 중에 1376


In [53]:
count = 0
for e in logest_text['text']:
    if '다' in e:
        count += 1
print(len(logest_text), '중에', count)

2381 중에 2069


## 문장 혹은 단어 추출

### 교수

In [186]:
def prof_eval_sen(row):
    text = row['text']
    
    if '.' in text: 
        start = text.find("교수")
        end = text.find(".",start)
    
    elif '다' in text: 
        start = text.find("교수")
        end = text.find("다",start)
    
    elif '요' in text: 
        start = text.find("교수")
        end = text.find("요",start)
        
    elif '\n' in text: 
        start = text.find("교수")
        end = text.find("\n",start)
    
    elif '심' in text: 
        start = text.find("교수")
        end = text.find("심",start)
        
    elif '임' in text: 
        start = text.find("교수")
        end = text.find("임",start)

    else:
        return text
        
    return text[start:end]       

In [1194]:
total['prof_eval_sen'] = total.apply(lambda x: prof_eval_sen(x), axis=1)

In [49]:
print('전체', len(total), '중에', len(total.loc[lambda x: x['prof_eval_sen']!= '']),'이 교수평 있음.')

전체 14037 중에 9594 이 교수평 있음.


# 특성 추출

### 시험 유형

In [90]:
def exam_type_memo(row):
    text = row['text']
    
    if '외워야' in text:
        return 1
    
    elif '외우기' in text:
        return 1
    
    elif '외워서' in text:
        start = text.find('외워서')
        if '안' in text[start:start+10]:
            return 0
        else:
            return 1
    
    elif '외우면' in text:
        start = text.find('외우면')
        if '안' in text[start:start+10]:
            return 0
        else:
            return 1
    
    elif '암기' in text:
        start = text.find('암기')
        if '없' in text[start:start+10] or '안' in text[start:start+10]:
            return 0
        else:
            return 1
    else:
        return -1

In [1197]:
total['is_memo'] = total.apply(lambda x: exam_type_memo(x), axis=1)

In [92]:
def exam_type_ess(row):
    text = row['text']
    
    if '논술' in text or '글빨' in text or '필력' in text or '약술' in text or '서술' in text or '주관식' in text:
        if row['is_memo'] == 1:
            return 0
        else:
            return 1    
    else:
        return -1

In [1198]:
total['is_ess'] = total.apply(lambda x: exam_type_ess(x), axis=1)

In [1199]:
# 둘 중 하나라도 정보 있는데 나머지는 정보 없는 거 채워줌.
total.loc[(total['is_memo']==-1) & (total['is_ess']==1), 'is_memo'] = 0
total.loc[(total['is_memo']==1) & (total['is_ess']==-1), 'is_ess'] = 0

In [1200]:
print(len(total.loc[lambda x: (x['is_memo'] == -1) & (x['is_ess'] == -1)]))

22468


### 레포트

In [99]:
def exi_rep(row):
    text = row['text']
    
    if '레포트' in text:
        start = text.find('레포트')
        if '안' in text[start:start+10] or '없' in text[start:start+10]:
            return 0
        else:
            return 1
        
    elif '보고서' in text:
        start = text.find('보고서')
        if '안' in text[start:start+10] or '없' in text[start:start+10]:
            return 0
        else:
            return 1
        
    elif '레폿' in text:
        start = text.find('레폿')
        if '안' in text[start:start+10] or '없' in text[start:start+10]:
            return 0
        else:
            return 1   
    
    else:
        return -1

In [1201]:
total['is_report'] = total.apply(lambda x: exi_rep(x), axis=1)

### 강의력

In [101]:
def lec_eval(row):
    text = row['text']
    
    if '강의력' in text:
        start = text.find('강의력')
        
        sen = text[start:start+20]
        if '좋' in sen and '안' in sen: 
            return 0
        elif '좋' in sen or '뛰' in sen or '훌륭' in sen or '최고' in sen or '굿' in sen or '쩜' in sen or '쩔' in sen or \
        '쩐' in sen or '오짐' in sen or '오져' in sen or '오졌' in sen or '오진' in sen or '지림' in sen or '지려' in sen or\
        '지린' in sen:
            return 2
        elif ('안' in sen and '나쁘' in sen) or ('안' in sen and '나뻐' in sen) or ('안' in sen and '나쁨' in sen):
            return 1
        elif '괜찮' in sen or '무난' in sen or '평범' in sen or '들을' in sen or '나쁘지' in sen or '평타' in sen:
            return 1
        elif '나쁘' in sen or '나뻐' in sen or '나쁨' in sen  or '별로' in sen or '망' in sen or '좆' in sen\
            or '딸' in sen or '쓰레기' in sen or '헬' in sen:
            return 0
        else:
            return -1
    
    elif '강의' in text:
        start = text.find('강의')
        
        sen = text[start:start+20]
        if '잘' in sen and '못' in sen:
            return 0
        elif '잘' in sen:
            return 2
        elif '못' in sen:
            return 0
        else:
            return -1
    else:
            return -1         

In [1354]:
total['lec_pow'] = total.apply(lambda x: lec_eval(x), axis=1)

In [ ]:
temp = total[['lecture_name','professor_name','semester','year','is_memo', 'is_ess', 'is_report', 'lec_pow','prof_eval_sen']]

In [120]:
def info(row):
    return row['is_memo']+row['is_ess']+row['is_report']+row['lec_pow']

In [191]:
temp['info'] = temp.apply(lambda x: info(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [192]:
temp = temp.loc[lambda x: x['info']!=-4]

In [1121]:
def minus_to_nan_memo(row):
    if row['is_memo'] == -1:
        return None
    else:
        return row['is_memo']

In [1122]:
def minus_to_nan_ess(row):
    if row['is_ess'] == -1:
        return None
    else:
        return row['is_ess']

In [1123]:
def minus_to_nan_rep(row):
    if row['is_report'] == -1:
        return None
    else:
        return row['is_report']

In [1124]:
def minus_to_nan_lec(row):
    if row['lec_pow'] == -1:
        return None
    else:
        return row['lec_pow']

In [1204]:
temp['is_memo'] = temp.apply(lambda x: minus_to_nan_memo(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [1205]:
temp['is_ess'] = temp.apply(lambda x: minus_to_nan_ess(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [1206]:
temp['is_report'] = temp.apply(lambda x: minus_to_nan_rep(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [1207]:
temp['lec_pow'] = temp.apply(lambda x: minus_to_nan_lec(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### 강의 평균 구하기

In [1213]:
mean = temp.groupby(['lecture_name', 'professor_name'])[['is_memo', 'is_ess', 'is_report', 'lec_pow']].mean()

In [1215]:
mean.reset_index(inplace=True,drop=False)

In [1216]:
mean.columns=['lecture_name','professor_name','is_memo_mean','is_ess_mean', 'is_report_mean', 'lec_pow_mean']

In [1218]:
lec_mean = pd.merge(mean, temp[['lecture_name','professor_name','prof_eval_sen']], on=['lecture_name', 'professor_name'])

In [1150]:
lec_mean

,lecture_name,professor_name,is_memo_mean,is_ess_mean,is_report_mean,lec_pow_mean,prof_eval_sen
0,17세기영국시,서홍원,0.0,1.0,NaN,NaN,교수님이 설명하시고 학생들끼리 조별로 토론하고 토론한 내용 발표하는 식으로 진행되는...
1,19세기미국소설,양석원,NaN,NaN,1.0,2.0,교수님도 영문과에 좀 알려진 명교수님 중 하나라서 졸업 전에 들어보고 싶었다
2,19세기미국소설,양석원,NaN,NaN,1.0,2.0,교수님의 의견을 모두 타이핑해서 받아적은 후에 그대로 책에 넣고 복사하는 식+ 자신...
3,19세기미국소설,양석원,NaN,NaN,1.0,2.0,교수님이 말씀하신 내용을 잘 필기해놓고 간단한 역사적 문맥은 외워 놓으면 시험 볼 ...
4,19세기미국소설,양석원,NaN,NaN,1.0,2.0,교수님 똑똑하시고 쪽글 피드백도 잘해주시고 시험문제도 예상할만한 범위에서 나옴
5,19세기미국소설,양석원,NaN,NaN,1.0,2.0,교수님이 영문학교수님들 중에서는 강의력이 상당히 좋으신 편이신 데다가 은근 귀여우신...
6,20세기러시아문학,최건영,NaN,NaN,NaN,NaN,교수님도 좀 웃기심 ㅎㅎㅎ 근데이제 수업 잘 안하실거라고 하셨음 ㅠ\n시험없고 책읽...
7,20세기러시아문학,최건영,NaN,NaN,NaN,NaN,
8,20세기미국소설,문상영,0.0,1.0,1.0,NaN,
9,20세기미국소설,이광진,NaN,NaN,NaN,NaN,


# 감성분석
- 하려고 했으나 실패

In [225]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Twitter

In [227]:
kkma = Kkma()
komo = Komoran()
twit = Twitter()

In [258]:
def extract_n_adj(row):
    pos_list = kkma.pos(row['prof_eval_sen'])
    result = []
    if len(pos_list)==0:
        return ''

    for token in pos_list:
        if token[0]!='교수님' and (token[1] == 'MAG' or token[1] == 'VA'):
            result.append(token[0]+'/'+token[1])
    return result

In [1371]:
fin_save['pos_list'] = fin.apply(lambda x: extract_n_adj(x), axis=1)

In [519]:
twit.pos('좋으시다')

[('좋', 'Adjective'), ('으', 'PreEomi'), ('시', 'PreEomi'), ('다', 'Eomi')]

In [516]:
kkma.pos('좋으시다')

[('좋', 'VA'), ('으시', 'EPH'), ('다', 'EFN')]

In [382]:
lec_mean['prof_eval_sen'][2]

'교수님의 의견을 모두 타이핑해서 받아적은 후에 그대로 책에 넣고 복사하는 식+ 자신의 기막힌 생각을 짬뽕해서 적어내야만 합니다'

### 한국어 감성 분석 사전(KOSAC)

- csv 다운 받은 곳: https://github.com/mrlee23/KoreanSentimentAnalyzer<br>
- kosac: http://word.snu.ac.kr/kosac/corpus.php


**polarity**는 주관적 표현이 긍정적인 의미인지 부정적인 의미인지를 나타내주기 위한 것이다.<br> 
polarity: positive, negative, neutral, complex

In [411]:
check = pd.read_csv('polarity.csv', encoding='utf-8')

In [2]:
import pandas as pd

In [3]:
sent = pd.read_csv('polarity.csv', encoding='utf-8')

In [4]:
sent.head()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
1,가*/JKS;있/VV,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
2,가*/JKS;있/VV;었/EP,1,0.0,0.0,0.0,0.0,1.0,POS,1.0
3,가*/VV,3,0.0,0.0,0.0,0.0,1.0,POS,1.0
4,가*/VV;ㄴ다*/EF,1,0.0,0.0,0.0,0.0,1.0,POS,1.0


In [359]:
sent = sent[['ngram','max.value']]

In [335]:
def not_slash(row):
    if ';' not in row['ngram'] and ('MAG' in row['ngram'] or 'VA' in row['ngram']):
        return row['ngram']
    else:
        return ''

In [360]:
sent['ngram'] = sent.apply(lambda x: not_slash(x), axis=1)
sent = sent.loc[lambda x: (x['ngram']!='') ]

In [ ]:
sent = sent.set_index(sent['ngram'])
del sent['ngram']

In [375]:
sent_dic = sent['max.value'].to_dict()

In [405]:
def gram_to_sent(row):
    if len(row['pos_list']) == 0:
        return ''
    else:
        sent_sum = 0
        val_list = []
        for gram in row['pos_list']:
            try:
                sent = sent_dic[gram]
                if sent == 'NEG':
                    val = -1
                elif sent == 'POS':
                    val = 1
                elif sent == 'NEUT' or sent == 'COMP' or sent == 'None':
                    val = 0
                val_list.append(val)    
            except KeyError:
                continue
        sent_sum = sum(val_list)
        return sent_sum

In [402]:
sent_dic['별로/MAG'] = 'NEG'
sent_dic['기막히/VA'] = 'COMP'
sent_dic['상당히/MAG'] = 'COMP'
sent_dic['진짜/MAG'] = 'COMP'
sent_dic['되게/MAG'] = 'COMP'
sent_dic['엄청/MAG'] = 'COMP'

In [406]:
lec_mean['sent_total'] = lec_mean.apply(lambda x: gram_to_sent(x), axis=1)

In [410]:
# 이런 이유로 못함.
sent_dic['귀엽/VA']

'NEG'

In [412]:
check.loc[lambda x: x['ngram'] == '귀엽/VA']

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
1792,귀엽/VA,1,0.0,1.0,0.0,0.0,0.0,NEG,1.0


# 최종 Data frame 만들기

In [1222]:
df = pd.merge(lec_mean, total[['date','lecture_name','professor_name','semester','year','attendance',
                           'exam_times','grade','homework', 'mean_rate','team','prof_eval_sen']], on=['lecture_name','professor_name','prof_eval_sen'])

In [1224]:
df = df.fillna(-1)
df['semester'] = df['semester'].astype(int)

In [430]:
crs = pd.read_csv('yonsei_course.csv', encoding='utf-8')
crs.columns =['college', 'id', 'mileage_avg', 'mileage_max', 'mileage_min', 'lecture_name',
       'professor_name', 'room', 'semester', 'weight', 'year']

In [1226]:
fin = pd.merge(df, crs, on=['lecture_name','professor_name','year','semester'])

In [1227]:
pd.set_option('display.max_columns', None)

In [1230]:
del fin['date']
del fin['id']
del fin['weight']

In [1231]:
fin_save = fin.copy()

In [1232]:
fin = fin.fillna(-1)

In [1160]:
def exc_grad(row):
    if row['college'] == -1 or row['college'] == '???? ?????':
        return 'Null'
    
    elif '대학원' in row['college']:
        return ''
    else:
        return row['college']

In [1233]:
fin['college'] = fin.apply(lambda x: exc_grad(x), axis=1)

In [1235]:
fin = fin.loc[lambda x: x['college']!='']

In [1236]:
fin_save = fin.copy()

In [1177]:
fin = fin_save.copy()

In [1238]:
fin = fin.drop_duplicates(['lecture_name', 'professor_name'])

In [1240]:
fin_drop = fin.copy()

# 강의 추천!

### 베스트, 워스트

In [1253]:
rec = fin_drop.copy()

In [1091]:
def sel_major(row):
    if row['college'] in maj:
        return row['college']
    else:
        return ''

In [1168]:
rec['college'] = rec.apply(lambda x: sel_major(x), axis=1)

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [1181]:
def exc_song(row):
    if '진' in row['room'] or '자' in row['room']:
        return ''
    else:
        return row['room']

In [1254]:
rec = rec.fillna('NaN')

In [1255]:
rec['room'] = rec.apply(lambda x: exc_song(x), axis=1)

In [1256]:
rec = rec.loc[lambda x: (x['college']!='') & (x['year']>=2016) & (x['room']!='')]

### 교양

In [1245]:
def ref_sel(row):
    if '대학' in row['lecture_name'] or '기독교' in row['lecture_name'] or row['lecture_name'] == '글쓰기' \
    or 'HOLISTIC' in row['lecture_name'] or 'RC' in row['lecture_name']:  
        return ''
    else:
        return row['lecture_name']

5점이 너무 많아서 학점느님 조건 추가

In [1292]:
ref = rec.loc[lambda x: (x['college']=='학부대학')]
ref['lecture_name']= ref.apply(lambda x: ref_sel(x), axis=1)
ref = ref.loc[lambda x: x['grade']=='학점느님']
ref = ref.loc[lambda x: x['lecture_name']!='']
r=ref.sort_values(['mean_rate'], ascending=[True])
r.iloc[6]

C:\Users\nhm\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



lecture_name      고급실용문쓰기
professor_name        최영석
is_memo_mean            0
is_ess_mean             1
is_report_mean          1
lec_pow_mean           -1
prof_eval_sen            
semester                1
year                 2017
attendance           직접호명
exam_times             없음
grade                학점느님
homework               많음
mean_rate            3.29
team                   보통
college              학부대학
mileage_avg         13.75
mileage_max            36
mileage_min             1
room                백S608
Name: 56591, dtype: object

### 경제학부

In [1267]:
eco = rec.loc[lambda x: (x['college']=='상경대학 경제학부')].sort_values(['mean_rate'], ascending=[False])
eco.head(5)

,lecture_name,professor_name,is_memo_mean,is_ess_mean,is_report_mean,lec_pow_mean,prof_eval_sen,semester,year,attendance,exam_times,grade,homework,mean_rate,team,college,mileage_avg,mileage_max,mileage_min,room
140000,산업과전략,김상현,-1.0,-1.0,-1.0,-1.0,교수님 똑똑한 지식을 옆에서 조금씩 엿보는 느낌의 수업이나,1,2017,반영안함,두 번,학점느님,보통,5.00,없음,상경대학 경제학부,5.19,12,1,상본B109
69938,국제금융론,성태윤,1.0,0.0,1.0,-1.0,교수님이 약간 나이에 비해 옛날분이시다,1,2017,직접호명,한 번,학점느님,보통,5.00,없음,상경대학 경제학부,10.24,12,1,상본B120
151942,시장설계,박재옥,-1.0,-1.0,-1.0,-1.0,,2,2016,반영안함,두 번,학점느님,없음,5.00,없음,상경대학 경제학부,8.28,12,1,상본115
149768,수리경제학,박재옥,-1.0,-1.0,-1.0,-1.0,교수님이 질문을 잘 받아주셔서 정말 좋습니,1,2016,직접호명,두 번,비율 채워줌,없음,5.00,없음,상경대학 경제학부,9.02,18,1,상본B103
135456,법경제학,김철영,1.0,0.0,1.0,2.0,교수님이 웃고있지 않는 때를 본 적이 없어서 진짜 존경스러움,1,2017,전자출결,두 번,학점느님,보통,4.96,없음,상경대학 경제학부,11.43,12,1,상본B130


In [1305]:
eco.tail(5).iloc[4]

lecture_name          은행과이자
professor_name      김광환,장은종
is_memo_mean             -1
is_ess_mean              -1
is_report_mean           -1
lec_pow_mean             -1
prof_eval_sen     교수님이 없습니다
semester                  1
year                   2016
attendance             직접호명
exam_times              두 번
grade                매우 깐깐함
homework                 없음
mean_rate                 1
team                     없음
college           상경대학 경제학부
mileage_avg            2.57
mileage_max               7
mileage_min               1
room                  상별102
Name: 179297, dtype: object

### 공과대학 컴퓨터과학과

In [1271]:
com = rec.loc[lambda x: (x['college']=='공과대학 컴퓨터과학과')].sort_values(['mean_rate'], ascending=False)
com.iloc[6]

lecture_name      인간과컴퓨터인터페이스
professor_name            한탁돈
is_memo_mean               -1
is_ess_mean                -1
is_report_mean             -1
lec_pow_mean               -1
prof_eval_sen        교수님 감사합니
semester                    2
year                     2016
attendance               직접호명
exam_times                한 번
grade                    학점느님
homework                   많음
mean_rate                4.75
team                       많음
college           공과대학 컴퓨터과학과
mileage_avg              7.95
mileage_max                36
mileage_min                 1
room                    공A658
Name: 180543, dtype: object

In [1310]:
com.tail().iloc[4]

lecture_name      소프트웨어종합설계(1)
professor_name             송주석
is_memo_mean                -1
is_ess_mean                 -1
is_report_mean              -1
lec_pow_mean                -1
prof_eval_sen                 
semester                     1
year                      2017
attendance                직접호명
exam_times                  없음
grade                   매우 깐깐함
homework                    많음
mean_rate                 1.33
team                        많음
college            공과대학 컴퓨터과학과
mileage_avg               5.08
mileage_max                 36
mileage_min                  1
room                     공A528
Name: 149753, dtype: object

### 공과대학 정보산업공학과

In [1276]:
ind = rec.loc[lambda x: (x['college']=='공과대학 산업공학과') | (x['college']=='공과대학 정보산업공학과')].sort_values(['mean_rate'], ascending=[False])
ind.iloc[4]

lecture_name                                지능정보시스템개론
professor_name                                    김우주
is_memo_mean                                       -1
is_ess_mean                                        -1
is_report_mean                                     -1
lec_pow_mean                                       -1
prof_eval_sen     교수님! 수업 내용은 인공지능에 대한 간단한 개념을 배우는 것임
semester                                            1
year                                             2017
attendance                                       전자출결
exam_times                                        두 번
grade                                          비율 채워줌
homework                                           보통
mean_rate                                           5
team                                               보통
college                                    공과대학 산업공학과
mileage_avg                                      7.79
mileage_max                                        36
mileage_min                 

In [1320]:
ind.tail(6).iloc[0]

lecture_name      창업302: 기술혁신경영
professor_name              박희준
is_memo_mean                 -1
is_ess_mean                  -1
is_report_mean               -1
lec_pow_mean                 -1
prof_eval_sen                  
semester                      1
year                       2017
attendance                 전자출결
exam_times                  두 번
grade                      학점느님
homework                     보통
mean_rate                     3
team                         보통
college              공과대학 산업공학과
mileage_avg                7.51
mileage_max                  27
mileage_min                   2
room                      공A432
Name: 194658, dtype: object

### 상경대학 응용통계학

In [1282]:
sta = rec.loc[lambda x: x['college']=='상경대학 응용통계학'].sort_values(['mean_rate'], ascending=[False])
sta.iloc[4]

lecture_name          손해보험통계
professor_name           김현태
is_memo_mean              -1
is_ess_mean               -1
is_report_mean            -1
lec_pow_mean              -1
prof_eval_sen               
semester                   2
year                    2016
attendance              전자출결
exam_times               두 번
grade                   학점느님
homework                  보통
mean_rate               4.67
team                      없음
college           상경대학 응용통계학
mileage_avg             8.88
mileage_max               18
mileage_min                1
room                  상본B112
Name: 149765, dtype: object

In [1325]:
sta.tail(5).iloc[4]

lecture_name               통계학입문
professor_name               왕상미
is_memo_mean                  -1
is_ess_mean                   -1
is_report_mean                -1
lec_pow_mean                   0
prof_eval_sen                   
semester                       1
year                        2017
attendance                  직접호명
exam_times                   두 번
grade                     비율 채워줌
homework                      보통
mean_rate                   2.25
team                          없음
college               상경대학 응용통계학
mileage_avg                18.13
mileage_max                   36
mileage_min                    1
room              상본B110(상본B110)
Name: 581738, dtype: object

### 평점 좋은 강의는 어느 과에 많을까?

In [1333]:
highest = fin_drop.loc[lambda x: x['mean_rate']>=5]

In [1336]:
col_counts = highest['college'].value_counts()

In [1337]:
col_counts.drop('학부대학')

교육과학대학 체육교육학        54
이과대학 수학             31
공과대학 전기전자공학과        29
교육과학대학 스포츠응용산업학과    28
교육과학대학 스포츠레저학과      27
언더우드국제대학            25
문과대학 중어중문학          21
공과대학 기계공학과          17
경영대학 경영학과           15
문과대학 노어노문학          14
문과대학 철학             13
문과대학 국어국문학          13
문과대학 불어불문학          12
문과대학 영어영문학          12
언더우드국제대학 언더우드학부     12
문과대학 사학             11
교육과학대학 교육학부         11
문과대학 심리학            11
문과대학 독어독문학          10
음악대학 작곡과            10
공과대학 신소재공학과         10
사회과학대학 정치외교학        10
글로벌인재학부              9
이과대학 화학              9
공과대학 화공생명공학과         8
이과대학 물리학             7
공과대학 건축공학과           7
사회과학대학 문화인류학과        7
이과대학 대기과학            7
공과대학 컴퓨터과학과          7
                    ..
사회과학대학 사회학           3
공과대학 토목·환경공학과        3
사회과학대학 사회복지학         3
공과대학 글로벌융합공학부        3
공과대학 산업공학과           3
교목실                  3
언더우드국제대학 융합사회과학부     3
공학교육혁신센터             2
언더우드국제대학 테크노아트학부     2
학부연계전공               2
상경대학 응용통계학           2
공과대학 도시공학과           2
교육과학대학 스포츠레

### 학점 잘 주는 교수님이 많을까?

In [1338]:
pro = fin_drop.copy()

In [1339]:
pro = pro.drop_duplicates('professor_name')

In [1340]:
grade_counts = rec['grade'].value_counts()

In [1341]:
grade_counts

비율 채워줌    995
학점느님      876
매우 깐깐함    267
F 폭격기      14
Name: grade, dtype: int64

과연 대망의 F 폭격기는...? 

In [1061]:
f = rec.loc[lambda x: x['grade']=='F 폭격기']
f

,lecture_name,professor_name,grade,mean_rate,mileage_avg,year,college,room
6640,전자기학(1),김덕영,F 폭격기,2.00,11.13,2016,이과대학 물리학,과323(과323)
23785,유기화학,정진현,F 폭격기,2.53,NaN,2016,약학대학 약학과,의대131호
27355,도시학개론,임업,F 폭격기,3.00,NaN,2016,공과대학 도시공학과,I진C302
32108,GREAT BOOKS & DEBATE (국가의탄생과전쟁),문상석,F 폭격기,1.64,8.06,2016,사회과학대학 사회학,백S608
32511,유기화학(2),태진성,F 폭격기,1.50,5.13,2016,이과대학 화학,과325
32604,대학영어Ⅱ,왕그레이스,F 폭격기,2.25,NaN,2016,학부대학,I진A402
38219,문학과영화,임정택,F 폭격기,2.00,11.80,2016,문과대학 독어독문학,외526
38271,CREATIVE THINKING AND VISUALIZATION,주하영,F 폭격기,1.14,12.70,2016,언더우드국제대학,I진B206
39178,공학수학(2),박승경,F 폭격기,3.00,NaN,2016,이과대학 수학,공A653/공A528(공A669)
42305,해석학(1),황숙정,F 폭격기,1.00,17.75,2016,이과대학 수학,과B101(과B101)


In [1062]:
f[['lecture_name','professor_name']]

,lecture_name,professor_name
6640,전자기학(1),김덕영
23785,유기화학,정진현
27355,도시학개론,임업
32108,GREAT BOOKS & DEBATE (국가의탄생과전쟁),문상석
32511,유기화학(2),태진성
32604,대학영어Ⅱ,왕그레이스
38219,문학과영화,임정택
38271,CREATIVE THINKING AND VISUALIZATION,주하영
39178,공학수학(2),박승경
42305,해석학(1),황숙정


그러면 학회 사람들이 많이 속한 과는 어떨까!

In [1063]:
def inclu_ybig(row):
    if row['college'] in maj[1:]:
        return row['college']
    else:
        return ''

In [1346]:
ybig = pro.copy()

In [1347]:
ybig['college'] = ybig.apply(lambda x: inclu_ybig(x), axis=1)

In [1348]:
ybig = ybig.loc[lambda x: x['college']!='']

In [1349]:
ybig['grade'].value_counts()

비율 채워줌    70
학점느님      42
매우 깐깐함    28
F 폭격기      1
Name: grade, dtype: int64

학점느님과 매우 깐깐함의 학과 분포를 봐보자.

In [1350]:
ybig.groupby('grade')['college'].value_counts()

grade   college     
F 폭격기   상경대학 경제학부        1
매우 깐깐함  상경대학 경제학부       17
        공과대학 컴퓨터과학과      6
        공과대학 정보산업공학과     2
        상경대학 응용통계학       2
        공과대학 산업공학과       1
비율 채워줌  상경대학 경제학부       36
        공과대학 컴퓨터과학과     14
        상경대학 응용통계학      11
        공과대학 산업공학과       6
        공과대학 정보산업공학과     3
학점느님    상경대학 경제학부       17
        상경대학 응용통계학      11
        공과대학 컴퓨터과학과      8
        공과대학 정보산업공학과     5
        공과대학 산업공학과       1
Name: college, dtype: int64

경제학부는 빈익빈 부익부가 심하다.

# best & worst 강의의 token 분석

In [1361]:
fin.columns

Index(['lecture_name', 'professor_name', 'is_memo_mean', 'is_ess_mean',
       'is_report_mean', 'lec_pow_mean', 'prof_eval_sen', 'semester', 'year',
       'attendance', 'exam_times', 'grade', 'homework', 'mean_rate', 'team',
       'college', 'mileage_avg', 'mileage_max', 'mileage_min', 'room'],
      dtype='object')

In [1456]:
pos = fin_save.copy()
pos = pos[['lecture_name', 'professor_name','mean_rate','pos_list']]

In [1457]:
pos = pos.sort_values(['mean_rate'])

In [1459]:
pos = pos.fillna('')

2.5점 이하 4.5점 이상

In [1460]:
len(pos.loc[lambda x: x['pos_list']!=''])

2090

In [1431]:
pos['mean_rate'].mean()

3.7953320896066303

In [1472]:
worst = pos.loc[lambda x: x['mean_rate']<=4.0]
best = pos.loc[lambda x: x['mean_rate']>4.0]

In [1473]:
len(worst.loc[lambda x: x['pos_list']!=''])

1154

In [1474]:
len(best.loc[lambda x: x['pos_list']!=''])

936

In [802]:
from collections import Counter

In [1475]:
worst_tokens = Counter()

In [1476]:
for tokens in worst['pos_list'].tolist():
    if len(tokens) == 0:
        continue
    else:
        for token in tokens:
                worst_tokens[token] += 1

In [1477]:
worst_tokens.most_common(20)

[('좋/VA', 224),
 ('잘/MAG', 147),
 ('없/VA', 113),
 ('같/VA', 100),
 ('많/VA', 89),
 ('너무/MAG', 86),
 ('정말/MAG', 82),
 ('많이/MAG', 70),
 ('안/MAG', 68),
 ('매우/MAG', 46),
 ('진짜/MAG', 42),
 ('좀/MAG', 42),
 ('다/MAG', 40),
 ('그냥/MAG', 37),
 ('열심히/MAG', 34),
 ('재밌/VA', 31),
 ('힘들/VA', 30),
 ('엄청/MAG', 30),
 ('어렵/VA', 30),
 ('별로/MAG', 29)]

In [1478]:
worst_va = []

for token in worst_tokens.most_common():
    if 'VA' in token[0]:
        worst_va.append((token))

In [1481]:
worst_va[:10]

[('좋/VA', 224),
 ('없/VA', 113),
 ('같/VA', 100),
 ('많/VA', 89),
 ('재밌/VA', 31),
 ('힘들/VA', 30),
 ('어렵/VA', 30),
 ('그렇/VA', 18),
 ('크/VA', 16),
 ('쉽/VA', 16)]

In [1480]:
result = 0
for _,num in worst_va:
    result+=num
print(result)

922


In [1482]:
best_tokens = Counter()

In [1483]:
for tokens in best['pos_list'].tolist():
    if len(tokens) == 0:
        continue
    else:
        for token in tokens:
                best_tokens[token] += 1

In [1484]:
best_tokens.most_common(20)

[('좋/VA', 282),
 ('정말/MAG', 111),
 ('잘/MAG', 110),
 ('너무/MAG', 91),
 ('같/VA', 65),
 ('많/VA', 64),
 ('없/VA', 58),
 ('진짜/MAG', 56),
 ('많이/MAG', 55),
 ('재밌/VA', 40),
 ('매우/MAG', 36),
 ('다/MAG', 34),
 ('좀/MAG', 33),
 ('굉장히/MAG', 30),
 ('안/MAG', 26),
 ('엄청/MAG', 25),
 ('쉽/VA', 24),
 ('어렵/VA', 22),
 ('그냥/MAG', 20),
 ('별로/MAG', 20)]

In [1485]:
best_va = []

for token in best_tokens.most_common():
    if 'VA' in token[0]:
        best_va.append(token)

In [1486]:
best_va[:10]

[('좋/VA', 282),
 ('같/VA', 65),
 ('많/VA', 64),
 ('없/VA', 58),
 ('재밌/VA', 40),
 ('쉽/VA', 24),
 ('어렵/VA', 22),
 ('재미있/VA', 16),
 ('귀엽/VA', 15),
 ('그렇/VA', 14)]

In [1497]:
com_dic = dict()
for w_t, w_f in worst_va:
    com_dic[w_t] = [w_f]

In [1498]:
import numpy as np

In [1499]:
for b_t, b_f in best_va:
    if b_t not in com_dic:
        com_dic[b_t] = 'upper only,{}'.format(b_f)
    else:
        com_dic[b_t].append((str(b_f),np.abs(com_dic[b_t][0]-b_f)))